**This script extracts climate relevant indicators on a monthly basis for different aggregation layers as csv table.**

Per aggregation layer, the script exports
- 1 csv table including all climate indicators (except groundwater) starting in 2017
- 1 csv table including groundwater storage anomaly starting in 2003

Possible improvements:
- For hexagon aggregation - remove hexagons with only few pixels of cropland as aggregated values based on few pixels might be misleading.
- Add option to export seasonal anomalies for each indicator to be able to compare between years/seasons more easily.
- Calculate rainfall and SPI monthly composites "on the fly" (currently pre-calculated and images exported to ee assets in a seperate javascript script).
- Add soil moisture anomalies (currently monthly soil moisture is calculated without comparing to a baseline).
- Add Vegetation Health Index (VHI). VHI combines Vegetation Condition Index (VCI) and Temperature Condition Index (TCI) https://code.earthengine.google.com/30cdc809525be6bf0fd5e50b243ca6e0
- Add Standard Precipitation Evaporation Index (SPEI)





In [ ]:
#@title Install packages { display-mode: "form" }
%%capture
# !pip install geopAndas
!pip install mycolorpy
!pip install geemap

In [ ]:
# Import libraries
import numpy  as np
import datetime
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter, YearLocator
from matplotlib.cm import ScalarMappable
from mycolorpy import colorlist as mcp
from sklearn import datasets, linear_model
import statsmodels.api as sm
import sys
# from pprint import pprint
from google.colab import files
import folium
import geemap
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()
ee.Initialize(project='ee-reachsyriagee')

In [ ]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/drive') # /content/drive

Mounted at /drive


In [ ]:
# Change recursion limit to avoid running into error when generating groundwater monthly image collection, otherwise too many recursions to call eeobject. See https://www.geeksforgeeks.org/python-sys-setrecursionlimit-method/
# limit = sys.getrecursionlimit()
# print('Before changing, limit of stack =', limit) # Print the current limit
Newlimit = 10000 # New limit
sys.setrecursionlimit(Newlimit) # Using sys.setrecursionlimit() method to find the current recursion limit
limit = sys.getrecursionlimit()
# print('After changing, limit of stack =', limit) # Print the current limit

## Functions

In [ ]:
# Generate image collections from assets
def generateCollectionFromAssets(imageNameBase,startYear, startMonth, endYear, endMonth):
  imageNameList =  ee.List([]);

  for j in range(startYear, endYear+1):
    for i in range (1, 13):
      if (j == startYear and i < startMonth):
        continue
      elif (j == endYear and i > endMonth):
        break

      # if (j < endYear and j > startYear) or (j == endYear and i <= endMonth) or (j == startYear and i >= startMonth):
      file_name = imageNameBase + str(j) + "_" + str(i);
      # print(file_name)
      image = ee.Image(file_name).set("month", i).set("year", j).set('system:time_start',ee.Date.fromYMD(j, i, 1).millis());
      imageNameList = imageNameList.add(image)

  return ee.ImageCollection(imageNameList)

def generateCollectionCropland (imageNameBase, startYear, endYear):
  years = [x for x in range(startYear, endYear+1)]
  imageNameList =  ee.List([]);

  for i in range(len(years)):
    file_name = imageNameBase + str(years[i]-1) + "_" + str(years[i])
    image = ee.Image(file_name).set('system:time_start', ee.Date(ee.Number(years[i]).format()))
    imageNameList = imageNameList.add(image)

  return ee.ImageCollection(imageNameList)

def generateCollection_monthly_hist(imageNameBase):
 months = [1,2,3,4,5,6,7,8,9,10,11,12]
 imageNameList =  ee.List([])

 for i in months:
  file_name = imageNameBase + str(i)
  image = ee.Image(file_name).set('system:time_start', ee.Date.fromYMD(1981, i, 1).millis())
  imageNameList = imageNameList.add(image)

 return ee.ImageCollection(imageNameList)

def generateCollectionAnnual (imageNameBase, startYear, endYear):
  years = [x for x in range(startYear, endYear+1)]
  imageNameList =  ee.List([]);

  for i in range(len(years)):
    file_name = imageNameBase + str(years[i])
    image = ee.Image(file_name).set('system:time_start', ee.Date(ee.Number(years[i]).format()).millis())
    imageNameList = imageNameList.add(image)

  return ee.ImageCollection(imageNameList)

# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):

    try:
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)

    except:
        print("Could not display {}".format(name))

def bitwiseExtract(input, fromBit, toBit):
# https://spatialthoughts.com/2021/08/19/qa-bAnds-bitmasks-gee/
  maskSize = ee.Number(1).add(toBit).subtract(fromBit)
  mask = ee.Number(1).leftShift(maskSize).subtract(1)
  return input.rightShift(fromBit).bitwiseAnd(mask)

# Cloud masking for MODIS-8 day
def maskCloudsMODIS8(image):
  # Select the QA bAnd.
  QA = image.select('State')
  # Use all possible bits to mask clouds And snow
  cloudState = bitwiseExtract(QA, 0, 1).eq(0)
  cirrus = bitwiseExtract(QA, 8, 9).eq(0)
  cloudAlgo = QA.bitwiseAnd(1 << 10).eq(0)
  cloudShadow = QA.bitwiseAnd(1 << 2).eq(0)
  snow = QA.bitwiseAnd(1 << 12).eq(0)
  snowAlgo = QA.bitwiseAnd(1 << 15).eq(0)
  adjacentToCloud = QA.bitwiseAnd(1 << 13).eq(0)

  mask = cloudState.And(cirrus).And(cloudAlgo).And(cloudShadow).And(snow).And(snowAlgo).And(adjacentToCloud)

  return image.updateMask(mask)

# Functions to temporally aggregate image collections
def composite_monthly_median(imgCol, date_range):
  return ee.ImageCollection.fromImages(\
      date_range.map(lambda date: imgCol.filterDate(ee.Date(date), ee.Date(date).advance(1,'month')).median()\
                     .set('year', ee.Date(date).get('year'))\
                     .set('month', ee.Date(date).get('month'))\
                     .set('Date', ee.Date(date).format('YYYY-MM'))\
                     .set('system:time_start', ee.Date(date).millis())\
                     .set('number_images', imgCol.filterDate(ee.Date(date), ee.Date(date).advance(1,'month')).size())))

# def composite_monthly_mean(imgCol, date_range):
#   return ee.ImageCollection.fromImages(\
#       date_range.map(lambda date: imgCol.filterDate(ee.Date(date), ee.Date(date).advance(1,'month')).mean()\
#                      .set('year', ee.Date(date).get('year'))\
#                      .set('month', ee.Date(date).get('month'))\
#                      .set('Date', ee.Date(date).format('YYYY-MM'))\
#                      .set('system:time_start', ee.Date(date).millis())\
#                      .set('number_images', imgCol.filterDate(ee.Date(date), ee.Date(date).advance(1,'month')).size())))


def composite_monthly_mean(imgCol, date_range, bandName):

  def filter_imgCol (imgCol,date):
      return imgCol.filterDate(ee.Date(date), ee.Date(date).advance(1,'month'))

  def mean_image (imgCol, date):
    filtered_imgCol = filter_imgCol(imgCol, date)
    filtered_mean = filtered_imgCol.mean()
    filtered_size = filtered_imgCol.size()
    return filtered_mean.set('year', ee.Date(date).get('year'))\
      .set('month', ee.Date(date).get('month'))\
      .set('Date', ee.Date(date).format('YYYY-MM'))\
      .set('system:time_start', ee.Date(date).millis())\
      .set('number_images', filtered_size)

  def no_data_image (date):
    return ee.Image([-99999]).rename(bandName).set('year', ee.Date(date).get('year'))\
      .set('month', ee.Date(date).get('month'))\
      .set('Date', ee.Date(date).format('YYYY-MM'))\
      .set('system:time_start', ee.Date(date).millis())\
      .set('number_images', ee.Number(0))

  return ee.ImageCollection.fromImages(date_range.map(lambda date: ee.Algorithms.If(filter_imgCol(imgCol, date).size(), mean_image(imgCol, date), no_data_image(date))))


def make_historical_median_monthly(imgCol, months_list):
    return ee.ImageCollection.fromImages(\
      months_list.map(lambda month: imgCol.filter(ee.Filter.calendarRange(month,month,'month'))\
      .median().set('month', ee.Number(month))));
def make_historical_mean_monthly(imgCol, months_list):
    return ee.ImageCollection.fromImages(\
      months_list.map(lambda month: imgCol.filter(ee.Filter.calendarRange(month,month,'month'))\
      .mean().set('month', ee.Number(month))));

def map_calculate_anomaly_monthly(imgCol, monthly_averages):
  return imgCol.map(lambda img: ee.Algorithms.If(img.bandNames(),\
                                                  img.subtract(monthly_averages.filterMetadata('month', 'equals', img.get('month')).first()).copyProperties(img, img.propertyNames()).set('isNull', 'false'),\
                                                  ee.Image([0]).mask(0).copyProperties(img, img.propertyNames()).set('isNull', 'true')))
def map_calculate_anomaly_perc_monthly(imgCol, monthly_averages):
  return imgCol.map(lambda img: ee.Algorithms.If(img.bandNames(),\
                                                  img.subtract(monthly_averages.filterMetadata('month', 'equals', img.get('month')).first()).divide(monthly_averages.filterMetadata('month', 'equals', img.get('month')).first()).multiply(100).copyProperties(img, img.propertyNames()).set('isNull', 'false'),\
                                                  ee.Image([0]).mask(0).copyProperties(img, img.propertyNames()).set('isNull', 'true')))

# Other functions
def make_date_range_monthly(start, end):
  n_months = end.difference(start, 'month').round().subtract(1);
  range = ee.List.sequence(0, n_months,1);
  def make_datelist(n):
    return start.advance(n,'month')

  return range.map(make_datelist);


def convert_coll_to_clientDict(imgCol, propNames_list):
  n = len(propNames_list)
  propVals = imgCol.reduceColumns(reducer = ee.Reducer.toList().repeat(n), selectors = propNames_list).get('list');
  dict_EE = ee.Dictionary.fromLists(propNames_list, propVals);
  clientDict = dict_EE.getInfo();

  return clientDict

def add_reduceRegion_as_property_to_imgcol(imgcol, bandnames, reducer, scale, aoi):

  def addProp (img):
    img = img.select(bandnames)

    dictionary = img.reduceRegion(
      geometry = aoi,\
      reducer = reducer,\
      scale = scale,\
      maxPixels = 1e9,\
      tileScale = 16\
    )
    return img.set(dictionary).set('Date', ee.Image(img).date().format('YYYY-MM-dd'))
  return imgcol.map(addProp)

def convert_imgcol_to_df (imgcol, bandnames, reducer, scale, aoi):
  # input imgcol must have a time stamp, bandnames = [], reducer = ee.Reducer, scale = int, aoi = ee.Geometry()

  imgcol = add_reduceRegion_as_property_to_imgcol(imgcol, bandnames, reducer, scale, aoi)
  propNames = ['Date'] + bandnames # property names
  dict_ = convert_coll_to_clientDict(imgcol, propNames) # convert image collection to dict
  df = pd.DataFrame(dict_) # convert dict to df
  df['Date'] = pd.to_datetime(df['Date']) # convert object to datetime

  return df

def reduceProperty(feat, property):
  properties = feat.propertyNames()
  selectProperties = properties.filter(ee.Filter.eq('item', property))
  return feat.select(selectProperties)

def catImages(feature): return ee.Image.cat(feature.get('primary'), feature.get('secondary'));

# FORMAT to features with 3 properties (columns in the output table)

def aggregate_to_fc_flattened_mean(coll, fc, scale, start, end, property_selectors, variable_name):

    def map_func(image): # if reducing an image collection with only 1 band, the reduced property name will automatically be named after the ee.Reducer(), so 'mean' - if more than 1 band, it will use band names, e.g. NDVI and NDVI_anom
        reduced_regions = image.reduceRegions(collection=fc, reducer=ee.Reducer.mean(), scale=scale, tileScale=16).filter(ee.Filter.neq('mean', None)).select(property_selectors.add('mean'))

        def inner_map_func(f):
            date = image.get('system:time_start')
            date_format = ee.Date(date).format('YYYY-MM')
            id_combined = ee.String(f.get(id_column_name)).cat('_').cat(date_format)
            return f.set('Date', date_format).set(row_id, id_combined).setGeometry(None)

        mapped_regions = reduced_regions.map(inner_map_func)
        return mapped_regions

    # Filter collection by date
    coll_filtered = coll.filterDate(ee.Date(start), ee.Date(end).advance(1, 'month'))

    # Apply mapping function to the filtered collection
    mapped_collection = coll_filtered.map(map_func)

    # Flatten the mapped collection
    flattened_collection = mapped_collection.flatten()
    # Remove and change property names
    prop_names = flattened_collection.first().propertyNames().remove('system:index')
    prop_names_new = prop_names.replace('mean', variable_name)
    flattened_collection = flattened_collection.select(prop_names, prop_names_new)


    return flattened_collection

# Function to join feature collections with 1 variable each to each other
def join_fc(fc1, fc2, id_name, variable_name_fc2):
    joined = ee.Join.saveAll('matches').apply(
        primary=fc1,
        secondary=fc2.select([id_name, variable_name_fc2]),
        condition=ee.Filter.equals(
            leftField=id_name,
            rightField=id_name
        )
    ).map(lambda row: row.set(variable_name_fc2, ee.Feature(ee.List(row.get('matches')).get(0)).get(variable_name_fc2)))

    return joined
# printing
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   ITALIC = '\033[3m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

## Set update parameters

*   Define start and end date of time series to export (format: 'YYYY-mm-01')
*   last date is inclusive


**EXAMPLES**

**Climate indicators** (all except groundwater)

Update current table with May 2024:
*   `update_table_climate = True `
*   `start_date_export = '2024-05-01'; last_date_export = '2024-05-01'`

Update current table with March - May 2024:
*   `update_table_climate = True `
*   `start_date_export = '2024-03-01'; last_date_export = '2024-05-01'`

Create new table including full time series (useful if new indicators will be added or shapefile column names will change)
*   `update_table_climate = False `
*   `start_date_export = '2017-01-01'; last_date_export = ...   `

**Groundwater**

Create new table including full time series with new end date
*   `start_date_gws_export = '2003-03-01'; last_date_gws_export = '2023-11-01'`
    

   
     
    
    
  
     
    

In [ ]:
update_table_climate = True # True to add new data to exisitng csv table (update table) - False to create a new base csv table

# Climate indicators (all except groundwater)
start_date_export = '2024-04-01'
last_date_export = '2024-05-01' # inclusive

last_date_current_csv = '2024-03-01' # ignore if update_table_climate = False
first_date_current_csv = '2017-01-01'

# Groundwater (no option to update, create new table)
start_date_gws_export = '2003-02-01'
last_date_gws_export = '2023-11-01'

# Folder path that stores csv tables
gdrive_output_folder =  '/drive/My Drive/Climate_monitoring/climate_indicator_time_series_monthly/' #  to save updated climate indicator table (pd.dataframe) as CSV. Slash mandatory in the end of the string
gdrive_output_folder_new_table = 'climate_indicator_time_series_monthly' # to save new csv table, exported from ee.FeatureCollection as CSV. ee export function does not take folder path, only name of lowest sub folder


# Select AOIs (aggregation layers)
AOI_names = ['ADM1', 'ADM2', 'ADM3', 'Hex', 'Agri_ADM1', 'Agri_ADM2'] # list of AOI names to iterate through

id_column_name = 'MASTER_ID' # ID column name in all aggregation shapefiles (ee.FeatureCollections) that contains a unique ID for each polygon

indicator_names = ['NDVI', 'NDVI_anom', 'NDVI_anom_perc', 'LST', 'LST_anom', 'LST_anom_perc',
                  'Precip' , 'Precip_anom', 'Precip_anom_perc', 'Precip_hist_mean', 'SPI', 'SSM']

indicator_names_file_name = ['NDVI', 'LST', 'Precip', 'SPI', 'SSM']
file_name_base = '_'.join(str(s) for s in indicator_names_file_name)
print('File base name for climate indicators: ', file_name_base)

if update_table_climate:
  print('Check newest date in current tables')

  for i in AOI_names:

    # Fetch most recent csv table stored in gdrive
    file_name = i + '_' + file_name_base + '_' + first_date_current_csv + '_' + last_date_current_csv
    print('CSV file name: ', file_name)
    # Check last date in csv table (should be same as file ending)
    try:
      df = pd.read_csv(gdrive_output_folder + file_name + '.csv')
      print('Newest date in CSV: ', df[['Date', id_column_name]].dropna().iloc[-1]['Date'])
    except FileNotFoundError:
      print(color.BOLD + "File not found." + color.END)
      break

File base name for climate indicators:  NDVI_LST_Precip_SPI_SSM
Check newest date in current tables
CSV file name:  ADM1_NDVI_LST_Precip_SPI_SSM_2017-01-01_2024-03-01
Newest date in CSV:  2024-03
CSV file name:  ADM2_NDVI_LST_Precip_SPI_SSM_2017-01-01_2024-03-01
Newest date in CSV:  2024-03
CSV file name:  ADM3_NDVI_LST_Precip_SPI_SSM_2017-01-01_2024-03-01
Newest date in CSV:  2024-03
CSV file name:  Agri_ADM1_NDVI_LST_Precip_SPI_SSM_2017-01-01_2024-03-01
Newest date in CSV:  2024-03
CSV file name:  Agri_ADM2_NDVI_LST_Precip_SPI_SSM_2017-01-01_2024-03-01
Newest date in CSV:  2024-03


## Load from assets

### Feature collections

Feature collections (i.e., shapefiles) stored as Earth Engine assets serve as spatial aggregation layers or geometry to clip images with

In [ ]:
baseRepo = "users/reachsyriagee/"
folder = "Admin_Areas/modified_reach/"

# ADMINISTRATIVE AREAS
fc_admin1  = ee.FeatureCollection(baseRepo + folder + "admin1_RS_AOI").select(ee.List([id_column_name, 'ADM1_EN', 'ADM1_PCODE','region']));
fc_admin2  = ee.FeatureCollection(baseRepo + folder + "admin2_RS_AOI").select(ee.List([id_column_name, 'ADM2_EN', 'ADM2_PCODE', 'ADM1_EN', 'ADM1_PCODE','region' ]));
fc_admin3  = ee.FeatureCollection(baseRepo + folder + "admin3_RS_AOI").select(ee.List([id_column_name, 'NAME_EN', 'PCODE', 'ADM2_EN', 'ADM2_PCODE', 'ADM1_EN', 'ADM1_PCODE', 'region']));

# AGRO-ECOLOGICAL ZONES
# fc_agri_zones = ee.FeatureCollection(baseRepo + "Admin_Areas/Agricultural_Stability_Zones").map(lambda f: f.set({'Agri_zone': f.get('Agri_Zone'), 'Agri_zone_descr': f.get('Agri_Zone_'), 'id': f.get('Agri_Zone1')}))

# AGRO-ECOLOGICAL ZONES INTERSECTED W ADMIN AREAS
def renameAgriProps(f): return f.set({'Agri_zone': f.get('Agri_Zone'), 'Agri_zone_descr': f.get('Agri_Zone_')}) # rename column (property) names
fc_agri_zones_admin1 = ee.FeatureCollection(baseRepo + folder + "admin1_Intersect_Agri").map(renameAgriProps)
fc_agri_zones_admin2 = ee.FeatureCollection(baseRepo + folder + "admin2_Intersect_Agri").map(renameAgriProps)
# fc_agri_zones_NES_NWS = ee.FeatureCollection(baseRepo + folder + "NES_NWS_Intersect_agri_zones").map(renameAgriProps)

# HEXAGONS (clipped to NES and NWS)
fc_hex_75 = ee.FeatureCollection(baseRepo + folder + 'hex_final')

# Geometry to clip images with
clip_geom =  ee.FeatureCollection(baseRepo + 'Admin_Areas/NES_NWS_simplified_boundaries').first().geometry()

### Image collections

In [ ]:
imageNameBase_water = baseRepo + 'SY_Water/final/water_' # 10m Sentinel
imageNameBase_chirps = baseRepo + 'SY_Rainfall/SY_CHIRPS_'
imageNameBase_chirps_histMean =  baseRepo + 'SY_Rainfall/historical/SY_CHIRPS_historical_mean_monthly_sum_00_'; # 00_20, 00_19, 00_16
imageNameBase_spi = baseRepo +  'SY_SPI/SPI1/SY_SPI_'
imageNameBase_crop = baseRepo + 'SY_Cropland/UNOSAT_crop_woodVeg_masked_'
imageNameBase_groundwater = 'projects/ee-reachsyriagee/assets/SY_Groundwater/GLDAS_anomaly_monthly_base0409/SY_GWS_Anom_';
imgNameBase_groundwater_annual = 'projects/ee-reachsyriagee/assets/SY_Groundwater/GLDAS_anomaly_seasonal_mean/SY_GWS_anom_seasonal_mean_11_10_';

# Time parameters (different for groundwater collection)
startYear = 2017
startMonth = 1
endYear = int(datetime.strptime(last_date_export, "%Y-%m-%d").year) # integer
endMonth = int(datetime.strptime(last_date_export, "%Y-%m-%d").month) # integer

chirps_end_year_baseline = 2016 # 2020, 2019, or 2016

# Rainfall sum
rain_monthly = generateCollectionFromAssets(imageNameBase_chirps, startYear, startMonth, endYear, endMonth)
rain_monthly_historical = generateCollection_monthly_hist(imageNameBase_chirps_histMean + str(chirps_end_year_baseline)[2:] + '_')
# Join historical rainfall image collection (12 images) with monthly rainfall sum, so historical rainfall can be exported for each date
filter_month = ee.Filter.equals(leftField = 'month', rightField = 'month' );
innerJoin = ee.Join.inner();
rain_monthly_and_histo = ee.ImageCollection(innerJoin.apply(rain_monthly, rain_monthly_historical.select(['Precipitation'], ['Precipitation_hist_mean']), filter_month)).map(catImages);

# SPI
spi_monthly = generateCollectionFromAssets(imageNameBase_spi, startYear, startMonth, endYear, endMonth)

try:
  ee.Algorithms.ObjectType(rain_monthly.first()).getInfo() == 'Image'
  # print('All good. CHIRPS images are updated in Earth Engine assets for user defined months/time period to export.')
except:
  print('!!! CHIRPS images are not updated in Earth Engine assets for user defined months/time period to export.')

# # Surface water
# surface_water_monthly = generateCollectionFromAssets(imageNameBase_water, startYear, startMonth, endYear, endMonth).map(lambda image: ee.Image(image).selfMask().select(["classification"],["Water"]))

# Groundwater

# Start and end date for monthly groundwater (end date is inclusive)
startYear_gws = 2003
startMonth_gws = 2
endYear_gws = int(datetime.strptime(last_date_gws_export, "%Y-%m-%d").year) # integer
endMonth_gws = int(datetime.strptime(last_date_gws_export, "%Y-%m-%d").month) # integer

start_gws = datetime(startYear_gws, startMonth_gws, 1, 0, 0).strftime('%Y-%m-%d') # date string
end_gws = datetime(endYear_gws, endMonth_gws, 1, 0, 0).strftime('%Y-%m-%d') # date string

gws_anom_monthly = generateCollectionFromAssets(imageNameBase_groundwater, startYear_gws, startMonth_gws, endYear_gws, endMonth_gws)
# gws_anom_annual = generateCollectionAnnual(imgNameBase_groundwater_annual, 2004, 2022)

# Cropland extent
cropland_imgCol = generateCollectionCropland(imageNameBase_crop,startYear,2022)
# cropland18_19 = cropland_imgCol.filterDate('2019-01-01', '2020-01-01').first() # season 2018/2019
cropland19_20 = cropland_imgCol.filterDate('2020-01-01', '2021-01-01').first() # season 2019/2020
cropland20_21 = cropland_imgCol.filterDate('2021-01-01', '2022-01-01').first() # season 2020/2021
cropland21_22 = cropland_imgCol.filterDate('2022-01-01', '2023-01-01').first() # season 2021/2022
UNOSAT_cropland_combined = cropland19_20.add(cropland20_21).add(cropland21_22).gte(1)

# WaPor Land Cover
# 20 shrubland, # 30 grassland, # 41 cropland rainfed, # 42 cropland irrigated, # 43 cropland fallow, # 50 built up, # 60 bare/sparse vegetation, # 80 water, # 111, 116, 121, 126 tree cover
lc22 = ee.Image('users/reachsyriagee/WaPor/L2_SYR_LCC_22').set('year', 2022)

# RASTER MASKS
# Non-water mask - to be used for LST
lc22_mask = lc22.eq(80).Not() # lc22.eq(60).Or(lc22.eq(60)).Not()

# Crop mask - to be used for NDVI
crop_mask = UNOSAT_cropland_combined

In [ ]:
# gws_anom_monthly.sort('system:time_start', False).first() # last groundwater image

### AOI dictionary

In [ ]:
aoi_dict = {
  "ADM1" : {
    "AOI": fc_admin1,
    "property_selectors": ee.List([id_column_name] + ['ADM1_EN', 'ADM1_PCODE', 'region']),
    "aggr_scale": 1000,
  },
  "ADM2": {
    "AOI": fc_admin2,
    "property_selectors": ee.List([id_column_name] + ['ADM2_EN', 'ADM2_PCODE', 'ADM1_EN', 'ADM1_PCODE', 'region']),
    "aggr_scale": 1000
  },
  "ADM3": {
    "AOI": fc_admin3.filter(ee.Filter.inList('region', ['NES', 'NWS'])),
    "property_selectors": ee.List([id_column_name] + ['NAME_EN', 'PCODE', 'ADM2_EN', 'ADM2_PCODE', 'ADM1_EN', 'ADM1_PCODE', 'region']),
    "aggr_scale": 1000
  },
  "Hex": {
    "AOI": fc_hex_75,
    "property_selectors": ee.List([id_column_name]),
    "aggr_scale": 5000 # = 25sqkm
  },
  # "Agri" : {
  #   "AOI": fc_agri_zones,
  #   "property_selectors": ee.List([id_column_name] + ['Agri_zone', 'Agri_zone_descr']),
  #   "aggr_scale": 1000
  # },
  "Agri_ADM1" : {
    "AOI": fc_agri_zones_admin1,
    "property_selectors": ee.List([id_column_name] +['NAME_EN', 'PCODE', 'region', 'Agri_zone', 'Agri_zone_descr']),
    "aggr_scale": 1000
  },
  "Agri_ADM2" : {
    "AOI": fc_agri_zones_admin2,
    "property_selectors": ee.List([id_column_name] + ['NAME_EN', 'PCODE', 'ADM1_EN', 'ADM1_PCODE', 'region', 'Agri_zone', 'Agri_zone_descr']),
    "aggr_scale": 1000
  }
}

# aoi_dict["ADM3"]["AOI"].first() # first polygon (ee.Feature) of admin3 layer

In [ ]:
# Map cropland mask
centroid = clip_geom.centroid()
# Set start zoom
zoom = 7
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[centroid.getInfo()['coordinates'][1], centroid.getInfo()['coordinates'][0]], zoom_start=zoom)
my_map.add_ee_layer(lc22_mask.selfMask(), {'palette':['orange']}, 'FAO LC 2022 - everything but water')
my_map.add_ee_layer(crop_mask.selfMask(), {'palette':['yellow']}, 'Crop mask')
my_map.add_ee_layer(aoi_dict["ADM3"]["AOI"], {}, 'Admin 3')

# # Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

## Load MODIS and SMAP collections



*   Compiling MODIS NDVI and LST and SMAP collections on a monthly basis from March 2000 (MODIS)/ April 2015 (SMAP)







In [ ]:
# Define start and end date of MODIS raw image collection
# start_date = datetime(startYear, startMonth, 1, 0, 0).strftime('%Y-%m-%d')
start_date_str = '2000-03-01'
start_date = ee.Date(start_date_str)

end_date_str = (datetime.strptime(last_date_export, "%Y-%m-%d")+ relativedelta(months=1)).strftime("%Y-%m-%d") # last date export plus 1 month
end_date = ee.Date(end_date_str)

# Define start and end of baseline.
start_date_base = start_date_str; # first of Jan date of the first season to be included (e.g. 2000-01-01 for 2000/2001)
end_date_base = '2016-08-01'

start_year_base = int(start_date_base[:4])
end_year_base = int(end_date_base[:4])

print('MODIS SR and LST image collections')
print('Date filter - START: ', start_date_str, 'END: ', end_date_str)
print('end baseline:',end_date_base)
# both MODIS collections start end of Feb 2000
# MODIS Surface reflectance (SR) 8-day
modis_ndvi = ee.ImageCollection('MODIS/061/MYD09Q1').merge(ee.ImageCollection('MODIS/061/MOD09Q1'))\
  .filterDate(start_date, end_date)\
  .filterBounds(clip_geom)\
  .sort('system:time_start')\
  .map(lambda img: img.clip(clip_geom))\
  .map(maskCloudsMODIS8)\
  .map(lambda img: img.addBands(img.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).rename('NDVI').updateMask(crop_mask))).select(['NDVI'])

last_date = modis_ndvi.limit(1, 'system:time_start', False).first().date().format('YYYY-MM-dd')
print('Last image date SR: ',  last_date.getInfo())


# MODIS Land surface temperature (LST) and emissivity
modis_lst = ee.ImageCollection("MODIS/061/MOD11A1").merge(ee.ImageCollection("MODIS/061/MYD11A1"))\
  .filterDate(start_date, end_date)\
  .filterBounds(clip_geom)\
  .map(lambda img: img.addBands(img.select('LST_Day_1km').multiply(0.02).subtract(273.15).rename('Celsius').float()).clip(clip_geom).updateMask(lc22_mask))\
  .select(['Celsius'])

last_date = modis_lst.limit(1, 'system:time_start', False).first().date().format('YYYY-MM-dd')
print('Last image date MODIS LST: ',  last_date.getInfo())

# SMAP Soil moisture
start_date_ssm = '2015-04-01' # starts March 31st


smap_ssm = ee.ImageCollection("NASA/SMAP/SPL4SMGP/007")\
  .filterDate(start_date_ssm, end_date)\
  .select('sm_surface')\
  .map(lambda img: img.clip(clip_geom).updateMask(crop_mask))

print('')
print('SMAP Soil moisture daily image collection: ')
print('Date filter - START: ',start_date_ssm, 'END: ',end_date_str)
print('Last image date: ', smap_ssm.sort('system:time_start', False).first().date().format('YYYY-MM-dd').getInfo())


MODIS SR and LST image collections
Date filter - START:  2000-03-01 END:  2024-06-01
end baseline: 2016-08-01
Last image date SR:  2024-05-24
Last image date MODIS LST:  2024-05-31

SMAP Soil moisture daily image collection: 
Date filter - START:  2015-04-01 END:  2024-06-01
Last image date:  2024-05-31


## Make monthly composites
*   Filtering monthly collections from start date till end date  (`start_date_export` and `last_date_export` indicated by user in section "Set update parameters")

In [ ]:
date_range = make_date_range_monthly(start_date, end_date)
months = ee.List([x for x in range(1, 13)])

# NDVI
modis_ndvi_monthly = composite_monthly_median(modis_ndvi, date_range)# .filter(ee.Filter.neq('number_images', 0)) # Remove composites without any images/bands d--- do not filter out because with gaps monthly anomaly images cant be calculated
modis_ndvi_historical = make_historical_median_monthly(modis_ndvi.filterDate(start_date_base, end_date_base), months); # historical ndvi per month (12 images)
modis_ndvi_monthly_filtered = modis_ndvi_monthly.filterDate(ee.Date(start_date_export), end_date); # monthly collection filtered to anomaly time period of interest

# NDVI anomaly
modis_ndvi_monthly_anom = map_calculate_anomaly_monthly(modis_ndvi_monthly_filtered, modis_ndvi_historical);

# NDVI anomaly (%)
modis_ndvi_monthly_anom_perc = map_calculate_anomaly_perc_monthly(modis_ndvi_monthly_filtered, modis_ndvi_historical);

# LST
modis_lst_monthly = composite_monthly_mean(modis_lst, date_range, 'Celsius')
modis_lst_historical = make_historical_median_monthly(modis_lst.filterDate(start_date_base, end_date_base), months);
modis_lst_monthly_filtered = modis_lst_monthly.filterDate(ee.Date(start_date_export), end_date);

# LST anomaly
modis_lst_monthly_anom = map_calculate_anomaly_monthly(modis_lst_monthly_filtered, modis_lst_historical);
# LST anomaly (%)
modis_lst_monthly_anom_perc = map_calculate_anomaly_perc_monthly(modis_lst_monthly_filtered, modis_lst_historical);


# VCI


# Rainfall anomaly
rain_monthly_anom = map_calculate_anomaly_monthly(rain_monthly, rain_monthly_historical);

# Rainfall anomaly (%)
rain_monthly_anom_perc = map_calculate_anomaly_perc_monthly(rain_monthly, rain_monthly_historical);


# Surface soil moisture
date_range_ssm = make_date_range_monthly(ee.Date(start_date_ssm), ee.Date(end_date).advance(1,'month'))
ssm_monthly = composite_monthly_mean(smap_ssm, date_range_ssm, 'sm_surface')


# TCI


# VHI


# Check size  of image collection
print("Number of months to be exported for climate indicators: ")
modis_lst_monthly_anom.size()

# Check last images
# ssm_monthly.sort('system:time_start', False).first()
# modis_lst_monthly.sort('system:time_start', False).first()


Number of months to be exported for climate indicators: 


### Visualise selected month



*   Select date, e.g. `last_date_export` or manually defined within export time period e.g. `'2024-02-01'`
* Select aggregation AOI e.g. `'Hex'` or `'ADM3'`




In [ ]:
date_str = last_date_export # Select date: last_date_expor or string '2024-02-01'
AOI_name_test = 'ADM3' # Select aggregation layer: 'ADM1', 'ADM2', 'ADM3', 'Hex', 'Agri_ADM1', 'Agri_ADM2', 'Agri_NES_NWS'


AOI_test = aoi_dict[AOI_name_test]['AOI']
aggr_scale_test = aoi_dict[AOI_name_test]['aggr_scale']
print('Selected month: ', date_str)
print('Selected AOI: ', AOI_name_test)

Selected month:  2024-05-01
Selected AOI:  ADM3


In [ ]:
date = ee.Date(date_str)
# last_date_ssm = date.advance(-1,'month')

# Set start zoom
zoom = 9

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualisation parameters
vis_ndvi =  {"opacity":1,"bands":["NDVI"],"min":0,"max":1, "palette": ["white", "green"]}
vis_ndvi_anom =  {"opacity":1,"bands":["NDVI"],"min":-0.3,"max":0.3, "palette": ["red", "white", "green"]}
vis_ndvi_anom_perc =  {"opacity":1,"bands":["NDVI"],"min":-50,"max":50, "palette": ["red", "white", "green"]}
vis_ndvi_anom_perc_aggr = {"opacity":1,"bands":["constant"],"min":-30,"max":30, "palette": ["red", "white", "green"]}

vis_lst =  {"opacity":1,"bands":["Celsius"],"min":10,"max":40, "palette": ["white", "orange"]}
vis_lst_anom =  {"opacity":1,"bands":["Celsius"],"min":-10,"max":10, "palette": ["blue", "white", "orange"]}
vis_lst_anom_perc =  {"opacity":1,"bands":["Celsius"],"min":-50,"max":50, "palette": ["blue", "white", "orange"]}
vis_lst_anom_perc_aggr = {"opacity":1,"bands":["constant"],"min":-20,"max":20, "palette": ["blue","white", "orange"]}

vis_rain = {"opacity":1,"bands":["Precipitation"],"min":0,"max":20, "palette": ["white", "blue"]}
vis_rain_anom = {"opacity":1,"bands":["Precipitation"],"min":-20,"max":20, "palette": ["red", "white", "blue"]}
vis_rain_anom_perc = {"opacity":1,"bands":["Precipitation"],"min":-50,"max":50, "palette": ["red", "white", "blue"]}
vis_rain_anom_perc_aggr = {"opacity":1,"bands":["constant"],"min":-30,"max":30, "palette": ["red", "white", "blue"]}
vis_spi = {"opacity":1,"bands":["SPI"],"min":-2,"max":2, "palette": ["red", "white", "blue"]}

vis_ssm = {"opaciyeah of ty":1,"bands":["sm_surface"],"min":0,"max":0.6, "palette": ["brown", "white", "blue"]}
# Create a folium map object.
my_map = folium.Map(location=[centroid.getInfo()['coordinates'][1], centroid.getInfo()['coordinates'][0]], zoom_start=zoom)

# Select last images of image collections
img_ndvi = modis_ndvi_monthly.filterDate(date,date.advance(1, 'month')).first()
img_ndvi_anom = modis_ndvi_monthly_anom.filterDate(date,date.advance(1, 'month')).first()
img_ndvi_anom_perc = modis_ndvi_monthly_anom_perc.filterDate(date,date.advance(1, 'month')).first()

img_lst = modis_lst_monthly.filterDate(date,date.advance(1, 'month')).first()
img_lst_anom = modis_lst_monthly_anom.filterDate(date,date.advance(1, 'month')).first()
img_lst_anom_perc = modis_lst_monthly_anom_perc.filterDate(date,date.advance(1, 'month')).first()

img_rain = rain_monthly.filterDate(date,date.advance(1, 'month')).first()
img_rain_anom = rain_monthly_anom.filterDate(date,date.advance(1, 'month')).first()
img_rain_anom_perc = rain_monthly_anom_perc.filterDate(date,date.advance(1, 'month')).first()

img_spi = spi_monthly.filterDate(date,date.advance(1, 'month')).first()
img_ssm = ssm_monthly.filterDate(date,date.advance(1, 'month')).first()
# img_ssm = ssm_monthly.filterDate(last_date_ssm,last_date_ssm.advance(1, 'month')).first()

# Reduce each image to a feature collection
# fc_ndvi = img_ndvi.reduceRegions(collection=AOI_test, reducer=ee.Reducer.mean(), scale=aggr_scale_test, tileScale=16)
fc_ndvi_anom_perc = img_ndvi_anom_perc.reduceRegions(collection=AOI_test, reducer=ee.Reducer.mean(), scale=aggr_scale_test, tileScale=16)
# fc_lst = img_lst.reduceRegions(collection=AOI_test, reducer=ee.Reducer.mean(), scale=aggr_scale_test, tileScale=16)
fc_lst_anom_perc = img_lst_anom_perc.reduceRegions(collection=AOI_test, reducer=ee.Reducer.mean(), scale=aggr_scale_test, tileScale=16)
fc_rain_anom_perc = img_rain_anom_perc.reduceRegions(collection=AOI_test, reducer=ee.Reducer.mean(), scale=aggr_scale_test, tileScale=16)

# Paint on empty image
empty_img = ee.Image().byte();
# fc_ndvi_painted = empty_img.paint(featureCollection = fc_ndvi, color= 'mean')
fc_ndvi_anom_perc_painted = empty_img.paint(featureCollection = fc_ndvi_anom_perc, color= 'mean')
# fc_lst_painted = empty_img.paint(featureCollection = fc_lst, color= 'mean')
fc_lst_anom_perc_painted = empty_img.paint(featureCollection = fc_lst_anom_perc, color= 'mean')
fc_rain_anom_perc_painted = empty_img.paint(featureCollection = fc_rain_anom_perc, color= 'mean')

# Visualise layer
my_map.add_ee_layer(img_ndvi.select(['NDVI']), vis_ndvi, 'NDVI  ' +date_str)
my_map.add_ee_layer(img_ndvi_anom.select(['NDVI']), vis_ndvi_anom, 'NDVI anomaly ' + date_str)
my_map.add_ee_layer(fc_ndvi_anom_perc_painted, vis_ndvi_anom_perc_aggr, 'NDVI anomaly (%) aggregated ' + date_str)
my_map.add_ee_layer(img_ndvi_anom_perc.select(['NDVI']), vis_ndvi_anom_perc, 'NDVI anomaly (%) ' + date_str)

my_map.add_ee_layer(img_lst.select(['Celsius']), vis_lst, 'LST  ' + date_str)
my_map.add_ee_layer(img_lst_anom.select(['Celsius']), vis_lst_anom, 'LST anomaly ' + date_str)
my_map.add_ee_layer(fc_lst_anom_perc_painted, vis_lst_anom_perc_aggr, 'LST anomaly (%) aggregated ' + date_str)
my_map.add_ee_layer(img_lst_anom_perc.select(['Celsius']), vis_lst_anom_perc, 'LST anomaly (%) ' + date_str)

my_map.add_ee_layer(img_rain.select(['Precipitation']), vis_rain, 'Rainfall  ' + date_str)
my_map.add_ee_layer(img_rain_anom.select(['Precipitation']), vis_rain_anom, 'Rainfall anomaly ' + date_str)
my_map.add_ee_layer(fc_rain_anom_perc_painted, vis_rain_anom_perc_aggr, 'Rainfall anomaly (%) aggregated ' + date_str)
my_map.add_ee_layer(img_rain_anom_perc.select(['Precipitation']), vis_rain_anom_perc, 'Rainfall anomaly (%) ' +date_str)

my_map.add_ee_layer(img_spi.select(['SPI']), vis_spi, 'SPI image ' + date_str)

my_map.add_ee_layer(img_ssm.select(['sm_surface']), vis_ssm, 'Soil moisture image ' + date_str)
# my_map.add_ee_layer(img_ssm.select(['sm_surface']), vis_ssm, 'Soil moisture image ' + last_date_ssm.format('YYYY-MM-dd').getInfo())
# my_map.add_ee_layer(crop_mask.selfMask(), {'palette':['yellow']}, 'crop_mask')

my_map.add_ee_layer(AOI_test, {}, 'AOI')

# # Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

# Export

In [ ]:
row_id = 'ID_comb' # column name for ID that combines date and MASTER_ID and date = unique ID in resulting csv table

## Climate indicators
(all except groundwater)

`update_table_climate == True`

*   Spatially aggregates pixel values for indicated time period
*   Adds to the new rows to the current table (exports it as new file)
*   Computionally less heavy


`update_table_climae == False`
*   Spatially aggregates pixel values for indicated time period
*   Creates new table
*   Computionally heavier


In [ ]:
print("Update exisiting table? ", update_table_climate)

Update exisiting table?  True


**Test aggregation before joining indicator feature collections and running export**

In [ ]:
# Test aggregation before running export.
AOI_name_test = 'ADM1' # 'ADM1', 'ADM2', 'ADM3', 'Hex', 'Agri_ADM1', 'Agri_ADM2', 'Agri_NES_NWS'
image_collection = spi_monthly # e.g. spi_monthly, modis_ndvi_monthly
band_name = 'SPI'
AOI_test = aoi_dict[AOI_name_test]['AOI']
aggr_scale_test = aoi_dict[AOI_name_test]['aggr_scale']
ee_selected_props = aoi_dict[AOI_name_test]['property_selectors']


fc = aggregate_to_fc_flattened_mean(image_collection, AOI_test, aggr_scale_test, start_date_export, last_date_export, ee_selected_props, band_name)

try:
  print('First polygon of feature collection: ', fc.first().getInfo())
  print('Aggregation worked')
except:
  print('Aggregation failed')

First polygon of feature collection:  {'type': 'Feature', 'geometry': None, 'id': '87_00000000000000000000', 'properties': {'ADM1_EN': 'Aleppo (NES)', 'ADM1_PCODE': 'SY02', 'Date': '2024-04', 'ID_comb': '0_2024-04', 'MASTER_ID': 0, 'SPI': -0.3225655222673685, 'region': 'NES'}}
Aggregation worked


**Export**

In [ ]:
# Start and end date for monthly indicators (end date is inclusive)
start = start_date_export
end = last_date_export

print('Selected start date: ', start)
print('Selected end date: ', end)


for i in AOI_names:
  print(color.BOLD + i + color.END)

  ee_selected_props = aoi_dict[i]['property_selectors'] # ee.List
  AOI = aoi_dict[i]['AOI'].select(ee_selected_props) # ee.FeatureCollection
  aggr_scale = aoi_dict[i]['aggr_scale'] # integer

  # Aggregate pixel values to mean values for each polygon
  fc_ndvi_monthly = aggregate_to_fc_flattened_mean(modis_ndvi_monthly_filtered, AOI, aggr_scale, start, end, ee_selected_props, 'NDVI')
  fc_ndvi_monthly_anom = aggregate_to_fc_flattened_mean(modis_ndvi_monthly_anom, AOI, aggr_scale, start, end, ee_selected_props, 'NDVI_anom')
  fc_ndvi_monthly_anom_perc = aggregate_to_fc_flattened_mean(modis_ndvi_monthly_anom_perc, AOI, aggr_scale, start, end, ee_selected_props, 'NDVI_anom_perc')
  fc_lst_monthly = aggregate_to_fc_flattened_mean(modis_lst_monthly_filtered, AOI, aggr_scale, start, end, ee_selected_props, 'LST')
  fc_lst_monthly_anom = aggregate_to_fc_flattened_mean(modis_lst_monthly_anom, AOI, aggr_scale, start, end, ee_selected_props, 'LST_anom')
  fc_lst_monthly_anom_perc = aggregate_to_fc_flattened_mean(modis_lst_monthly_anom_perc, AOI, aggr_scale, start, end, ee_selected_props, 'LST_anom_perc')
  fc_rain_monthly = aggregate_to_fc_flattened_mean(rain_monthly, AOI, aggr_scale, start, end, ee_selected_props, 'Precip')
  fc_rain_monthly_anom = aggregate_to_fc_flattened_mean(rain_monthly_anom, AOI, aggr_scale, start, end, ee_selected_props, 'Precip_anom')
  fc_rain_monthly_anom_perc = aggregate_to_fc_flattened_mean(rain_monthly_anom_perc, AOI, aggr_scale, start, end, ee_selected_props, 'Precip_anom_perc')
  fc_rain_monthly_hist_mean = aggregate_to_fc_flattened_mean(rain_monthly_and_histo.select(['Precipitation_hist_mean']), AOI, aggr_scale, start, end, ee_selected_props, 'Precip_hist_mean')
  fc_spi_monthly = aggregate_to_fc_flattened_mean(spi_monthly, AOI, aggr_scale, start, end, ee_selected_props, 'SPI')
  fc_ssm_monthly = aggregate_to_fc_flattened_mean(ssm_monthly, AOI, aggr_scale, start, end, ee_selected_props, 'SSM')


  # Join the tables
  fc_joined = join_fc(fc_ndvi_monthly, fc_ndvi_monthly_anom, row_id, 'NDVI_anom') # Join NDVI anomaly feature collection to NDVI feature collection
  fc_joined = join_fc(fc_joined, fc_ndvi_monthly_anom_perc, row_id, 'NDVI_anom_perc')
  fc_joined = join_fc(fc_joined, fc_lst_monthly, row_id, 'LST')
  fc_joined = join_fc(fc_joined, fc_lst_monthly_anom, row_id, 'LST_anom')
  fc_joined = join_fc(fc_joined, fc_lst_monthly_anom_perc, row_id, 'LST_anom_perc')
  fc_joined = join_fc(fc_joined, fc_rain_monthly, row_id, 'Precip')
  fc_joined = join_fc(fc_joined, fc_rain_monthly_anom, row_id, 'Precip_anom')
  fc_joined = join_fc(fc_joined, fc_rain_monthly_anom_perc, row_id, 'Precip_anom_perc')
  fc_joined = join_fc(fc_joined, fc_rain_monthly_hist_mean, row_id, 'Precip_hist_mean')
  fc_joined = join_fc(fc_joined, fc_spi_monthly, row_id, 'SPI')
  fc_joined = join_fc(fc_joined, fc_ssm_monthly, row_id, 'SSM')

  property_names = ['Date', row_id] + ee_selected_props.getInfo() # list
  # print('Property names: ', property_names)
  column_names_new_data = property_names + indicator_names
  # print('Column names new table', column_names_new_data)

  if update_table_climate:
    try:
      # Fetch most recent csv table stored in gdrive
      file_name = i + '_' + file_name_base + '_' + first_date_current_csv + '_' + last_date_current_csv + '.csv'
      print('Current CSV file name: ', file_name)
      # Check last date in csv table (should be same as file ending)
      df = pd.read_csv(gdrive_output_folder + file_name )
      print('Newest date in current CSV: ', df[['Date', id_column_name]].dropna().iloc[-1]['Date'])

    except FileNotFoundError:
      print(color.BOLD + "CSV file not found." + color.END)
      break

    # Get column names of current csv table
    column_names = list(df)
    # print('Column names current table: ', column_names)

    if set(column_names_new_data) == set(column_names):
      # print('Column names are identical')

      print(f'Number of rows of the current table: {len(df)}')
      df_new = df.copy()
      #  Add new rows to table - convert ee.FeatureCollection to pd.df
      dict_ = convert_coll_to_clientDict(fc_joined, column_names_new_data)
      df_data_to_add = pd.DataFrame(dict_)
      print(f'Number of rows of the data to add: {len(df_data_to_add)} ')
      df_merged = pd.concat([df_new, df_data_to_add])#.reset_index().drop(['index'], axis=1)
      print(f'Number of rows of merged table: {len(df_merged)}')

      file_name_new = i + '_' + file_name_base + '_' + first_date_current_csv + '_' + end + '.csv'
      print('New CSV file name: ', file_name_new)

      # # Export csv to local drive
      # df_merged.to_csv(file_name_new)
      # files.download(file_name_new)

      # Export csv to Google Drive
      df_merged.to_csv(gdrive_output_folder + file_name_new, index=False)
      print('Updated CSV table saved to gdrive folder ', color.ITALIC + gdrive_output_folder + color.END)
      print(' ')

    else:
      print(color.BOLD + 'Column names of new and current (in gdrive) table are different' + color.END)
      break

  else:

    file_name_new = i + '_' + file_name_base + '_' + start + '_' + end
    print('CSV file name: ', file_name_new)

    # Export table as csv to drive
    task = ee.batch.Export.table.toDrive(
        collection = fc_joined,
        description = file_name_new,
        folder = gdrive_output_folder_new_table,
        fileFormat = 'CSV',
        selectors = column_names_new_data,
    )
    task.start()
    print('New base CSV table exportig to gdrive folder: ' + color.ITALIC + gdrive_output_folder_new_table + color.END)
    print(' ')

Selected start date:  2024-04-01
Selected end date:  2024-05-01
ADM1
Current CSV file name:  ADM1_NDVI_LST_Precip_SPI_SSM_2017-01-01_2024-03-01.csv
Newest date in current CSV:  2024-03
Number of rows of the current table: 522
Number of rows of the data to add: 12 
Number of rows of merged table: 534
New CSV file name:  ADM1_NDVI_LST_Precip_SPI_SSM_2017-01-01_2024-05-01.csv
Updated CSV table saved to gdrive folder  /drive/My Drive/Climate_monitoring/climate_indicator_time_series_monthly/
 
ADM2
Current CSV file name:  ADM2_NDVI_LST_Precip_SPI_SSM_2017-01-01_2024-03-01.csv
Newest date in current CSV:  2024-03
Number of rows of the current table: 2088
Number of rows of the data to add: 48 
Number of rows of merged table: 2136
New CSV file name:  ADM2_NDVI_LST_Precip_SPI_SSM_2017-01-01_2024-05-01.csv
Updated CSV table saved to gdrive folder  /drive/My Drive/Climate_monitoring/climate_indicator_time_series_monthly/
 
ADM3
Current CSV file name:  ADM3_NDVI_LST_Precip_SPI_SSM_2017-01-01_2024-

## Groundwater

Time series starts in 2003

**Test aggregation before joining indicator feature collections and running export**

In [ ]:
# Test aggregation before running export.
AOI_name_test = 'ADM1' # 'ADM1', 'ADM2', 'ADM3', 'Hex', 'Agri_ADM1', 'Agri_ADM2', 'Agri_NES_NWS'

AOI_test = aoi_dict[AOI_name_test]['AOI']
aggr_scale_test = aoi_dict[AOI_name_test]['aggr_scale']
ee_selected_props = aoi_dict[AOI_name_test]['property_selectors']

fc = aggregate_to_fc_flattened_mean(gws_anom_monthly, AOI_test, aggr_scale_test, start_date_gws_export, last_date_gws_export, ee_selected_props, 'GWS')

try:
  print('First polygon of feature collection: ', fc.first().getInfo())
  print('Aggregation worked')
except:
  print('Aggregation failed')

First polygon of feature collection:  {'type': 'Feature', 'geometry': None, 'id': '0_00000000000000000000', 'properties': {'ADM1_EN': 'Aleppo (NES)', 'ADM1_PCODE': 'SY02', 'Date': '2003-02', 'GWS': 95.90199559719647, 'ID_comb': '0_2003-02', 'MASTER_ID': 0, 'region': 'NES'}}
Aggregation worked


**Export**

In [ ]:
# Start and end date for monthly indicators (end date is inclusive)
start = start_date_gws_export
end = last_date_gws_export

print('Selected start date: ', start)
print('Selected end date: ', end)


for i in AOI_names:
  print(color.BOLD + i + color.END)

  ee_selected_props = aoi_dict[i]['property_selectors'] # ee.List
  AOI = aoi_dict[i]['AOI'].select(ee_selected_props) # ee.FeatureCollection
  aggr_scale = aoi_dict[i]['aggr_scale'] # integer

  # Aggregate pixel values to mean values for each polygon
  fc_gws_monthly = aggregate_to_fc_flattened_mean(gws_anom_monthly, AOI, aggr_scale, start_gws, end_gws, ee_selected_props, 'GWS')

  property_names = ['Date', row_id] + ee_selected_props.getInfo() # list
  # print('Property names: ', property_names)
  column_names_new_data = property_names + ['GWS']
  # print('Column names new table', column_names_new_data)


  file_name = i + '_' + 'GWS' + '_' + start + '_' + end
  print('CSV file name: ', file_name)

  # Export table as csv to drive
  task = ee.batch.Export.table.toDrive(
      collection = fc_gws_monthly,
      description = file_name,
      folder = gdrive_output_folder_new_table,
      fileFormat = 'CSV',
      selectors = column_names_new_data,
  )
  task.start()
  print('CSV table exportig to gdrive folder: ' + color.ITALIC + gdrive_output_folder_new_table + color.END)
  print(' ')


Selected start date:  2003-02-01
Selected end date:  2023-03-01
ADM1
CSV file name:  ADM1_GWS_2003-02-01_2023-03-01
CSV table exportig to gdrive folder: climate_indicator_time_series_monthly
 
ADM2
CSV file name:  ADM2_GWS_2003-02-01_2023-03-01
CSV table exportig to gdrive folder: climate_indicator_time_series_monthly
 
ADM3
CSV file name:  ADM3_GWS_2003-02-01_2023-03-01
CSV table exportig to gdrive folder: climate_indicator_time_series_monthly
 
Hex
CSV file name:  Hex_GWS_2003-02-01_2023-03-01
CSV table exportig to gdrive folder: climate_indicator_time_series_monthly
 
Agri_ADM1
CSV file name:  Agri_ADM1_GWS_2003-02-01_2023-03-01
CSV table exportig to gdrive folder: climate_indicator_time_series_monthly
 
Agri_ADM2
CSV file name:  Agri_ADM2_GWS_2003-02-01_2023-03-01
CSV table exportig to gdrive folder: climate_indicator_time_series_monthly
 
